# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [17]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url)
data.status_code ## if it returns 200, it means that it is working properly

200

In [4]:
data = requests.get(api_url).json() #lets turn it into json
data #we want specifically the one year price return 

{'companyName': 'Apple Inc',
 'marketcap': 2539295184728,
 'week52high': 158.27,
 'week52low': 104.64,
 'week52highSplitAdjustOnly': 159.75,
 'week52lowSplitAdjustOnly': 106.4,
 'week52change': 0.3412448693875498,
 'sharesOutstanding': 17105207192,
 'float': 0,
 'avg10Volume': 96613261,
 'avg30Volume': 79307910,
 'day200MovingAvg': 138.23,
 'day50MovingAvg': 154.48,
 'employees': 154192,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.881830165961749,
 'dividendYield': 0.005719022805796917,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-22',
 'nextEarningsDate': '2021-10-15',
 'peRatio': 29.79611234680804,
 'beta': 1.4970390139531184,
 'maxChangePercent': 58.496584944516,
 'year5ChangePercent': 4.594019724658255,
 'year2ChangePercent': 1.7867654445043109,
 'year1ChangePercent': 0.3110483761352406,
 'ytdChangePercent': 0.1298611882935447,
 'month6ChangePercent': 0.207567989094692,
 'month3ChangePercent': 0.15532130806955913,
 'month1ChangePercent': -0.0004904104461450735,
 'day30ChangePerce

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
#to parse a JSON object, you pass in square brackets, and then the key of the dictonary
data["year1ChangePercent"]

0.3110483761352406

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings: 
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]["price"],
            data[symbol]["stats"]["year1ChangePercent"],
            "N/A"
        ],
        index = my_columns), 
        ignore_index = True
        )
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,178.280,0.773195,N/A
1,AAL,19.499,0.496255,N/A
2,AAP,209.960,0.336902,N/A
3,AAPL,149.450,0.31221,N/A
4,ABBV,107.300,0.247372,N/A
...,...,...,...,...
500,YUM,128.320,0.399451,N/A
501,ZBH,150.123,0.0189868,N/A
502,ZBRA,596.690,1.30033,N/A
503,ZION,58.750,0.958224,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values("One-Year Price Return", ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,31.010,2.38445,N/A
1,275,LB,80.820,2.29871,N/A
2,317,MRO,12.352,1.95006,N/A
3,175,FANG,82.200,1.78035,N/A
4,111,COTY,8.520,1.65781,N/A
5,357,OXY,27.358,1.64352,N/A
6,195,FTNT,315.000,1.57883,N/A
7,29,AMAT,140.350,1.54068,N/A
8,253,IVZ,25.700,1.53221,N/A
9,23,ALB,229.520,1.45079,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [53]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    
    try: 
        float(portfolio_size)
    except ValueError: 
        print("That is not a number! \nPlease try again:")
        portfolio_size = input("Enter the size of your portfolio: ")
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 100000000
100000000


In [54]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Price"])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,31.010,2.38445,64495
1,275,LB,80.820,2.29871,24746
2,317,MRO,12.352,1.95006,161917
3,175,FANG,82.200,1.78035,24330
4,111,COTY,8.520,1.65781,234741
5,357,OXY,27.358,1.64352,73104
6,195,FTNT,315.000,1.57883,6349
7,29,AMAT,140.350,1.54068,14250
8,253,IVZ,25.700,1.53221,77821
9,23,ALB,229.520,1.45079,8713


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [55]:
hqm_columns = [
    "Ticker", 
    "Price", 
    "Number of Shares to Buy", 
    "One-Year Price Return", 
    "One-Year Return Percentile", 
    "Six-Month Price Return", 
    "Six-Month Return Percentile", 
    "Three-Month Price Return", 
    "Three-Month Return Percentile", 
    "One-Month Price Return", 
    "One-Month Return Percentile",
    "HQM Score"
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]["price"], 
            "N/A",
            data[symbol]["stats"]["year1ChangePercent"],
            "N/A",
            data[symbol]["stats"]["month6ChangePercent"], 
            "N/A",
            data[symbol]["stats"]["month3ChangePercent"], 
            "N/A",
            data[symbol]["stats"]["month1ChangePercent"], 
            "N/A",
            "N/A"
        ],
        index = hqm_columns), 
        ignore_index = True
        )
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.182,N/A,0.750564,N/A,0.408143,N/A,0.207659,N/A,0.095756,N/A,N/A
1,AAL,19.858,N/A,0.484055,N/A,-0.238791,N/A,-0.153867,N/A,-0.0241813,N/A,N/A
2,AAP,209.060,N/A,0.330436,N/A,0.149145,N/A,0.0329785,N/A,-0.00109672,N/A,N/A
3,AAPL,151.616,N/A,0.304288,N/A,0.209431,N/A,0.155294,N/A,-0.00048661,N/A,N/A
4,ABBV,107.870,N/A,0.254101,N/A,0.000339647,N/A,-0.0596103,N/A,-0.0754994,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,133.480,N/A,0.40555,N/A,0.202345,N/A,0.0907379,N/A,-0.0452063,N/A,N/A
501,ZBH,150.680,N/A,0.0184539,N/A,-0.0968771,N/A,-0.078866,N/A,-0.00056922,N/A,N/A
502,ZBRA,585.540,N/A,1.28606,N/A,0.220505,N/A,0.154705,N/A,0.0179165,N/A,N/A
503,ZION,58.610,N/A,0.927216,N/A,0.017738,N/A,0.0728938,N/A,0.0485821,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [56]:
time_periods = [
                "One-Year",
                "Six-Month",
                "Three-Month",
                "One-Month"
                ]

In [57]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

In [58]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.182,N/A,0.750564,85.1485,0.408143,95.2475,0.207659,93.8614,0.095756,96.4356,N/A
1,AAL,19.858,N/A,0.484055,65.5446,-0.238791,1.9802,-0.153867,5.14851,-0.0241813,36.8317,N/A
2,AAP,209.060,N/A,0.330436,46.5347,0.149145,68.1188,0.0329785,59.802,-0.00109672,53.8614,N/A
3,AAPL,151.616,N/A,0.304288,43.1683,0.209431,77.4257,0.155294,88.7129,-0.00048661,55.2475,N/A
4,ABBV,107.870,N/A,0.254101,37.0297,0.000339647,31.4851,-0.0596103,24.7525,-0.0754994,9.90099,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,133.480,N/A,0.40555,55.6436,0.202345,76.2376,0.0907379,76.4356,-0.0452063,21.5842,N/A
501,ZBH,150.680,N/A,0.0184539,10.099,-0.0968771,13.4653,-0.078866,20.198,-0.00056922,54.8515,N/A
502,ZBRA,585.540,N/A,1.28606,96.6337,0.220505,79.2079,0.154705,88.3168,0.0179165,68.7129,N/A
503,ZION,58.610,N/A,0.927216,91.2871,0.017738,35.6436,0.0728938,72.0792,0.0485821,85.7426,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [59]:
from statistics import mean 

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f"{time_period} Return Percentile"])
    hqm_dataframe.loc[row, "HQM Score"] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,178.182,N/A,0.750564,85.1485,0.408143,95.2475,0.207659,93.8614,0.095756,96.4356,92.6733
1,AAL,19.858,N/A,0.484055,65.5446,-0.238791,1.9802,-0.153867,5.14851,-0.0241813,36.8317,27.3762
2,AAP,209.060,N/A,0.330436,46.5347,0.149145,68.1188,0.0329785,59.802,-0.00109672,53.8614,57.0792
3,AAPL,151.616,N/A,0.304288,43.1683,0.209431,77.4257,0.155294,88.7129,-0.00048661,55.2475,66.1386
4,ABBV,107.870,N/A,0.254101,37.0297,0.000339647,31.4851,-0.0596103,24.7525,-0.0754994,9.90099,25.7921
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,133.480,N/A,0.40555,55.6436,0.202345,76.2376,0.0907379,76.4356,-0.0452063,21.5842,57.4752
501,ZBH,150.680,N/A,0.0184539,10.099,-0.0968771,13.4653,-0.078866,20.198,-0.00056922,54.8515,24.6535
502,ZBRA,585.540,N/A,1.28606,96.6337,0.220505,79.2079,0.154705,88.3168,0.0179165,68.7129,83.2178
503,ZION,58.610,N/A,0.927216,91.2871,0.017738,35.6436,0.0728938,72.0792,0.0485821,85.7426,71.1881


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [60]:
hqm_dataframe.sort_values("HQM Score", ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.560,N/A,2.34356,99.802,0.848791,100,0.218954,94.6535,0.0831477,95.8416,97.5743
1,PWR,123.020,N/A,1.3199,97.0297,0.343986,91.4851,0.294365,97.2277,0.221469,99.802,96.3861
2,IT,324.290,N/A,1.37873,97.4257,0.739534,99.802,0.358391,99.0099,0.039929,81.9802,94.5545
3,NVDA,230.090,N/A,0.736149,83.5644,0.715856,99.604,0.259307,96.6337,0.11042,97.6238,94.3564
4,A,178.182,N/A,0.750564,85.1485,0.408143,95.2475,0.207659,93.8614,0.095756,96.4356,92.6733
5,INTU,582.460,N/A,0.802408,86.7327,0.458926,96.8317,0.226114,95.6436,0.0606672,91.0891,92.5743
6,NTAP,95.890,N/A,1.12794,95.2475,0.375705,94.0594,0.118133,81.7822,0.139593,99.0099,92.5248
7,SNPS,340.300,N/A,0.609278,77.0297,0.451502,96.6337,0.277043,97.0297,0.147547,99.2079,92.4752
8,KEYS,184.150,N/A,0.874878,89.1089,0.318453,89.901,0.220409,95.0495,0.0820152,95.4455,92.3762
9,RMD,291.840,N/A,0.696975,81.3861,0.575799,98.6139,0.236142,95.8416,0.059628,90.495,91.5842


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [61]:
portfolio_input()

Enter the size of your portfolio: 100000000


In [62]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[i, "Price"])
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.560,24521,2.34356,99.802,0.848791,100,0.218954,94.6535,0.0831477,95.8416,97.5743
1,PWR,123.020,16257,1.3199,97.0297,0.343986,91.4851,0.294365,97.2277,0.221469,99.802,96.3861
2,IT,324.290,6167,1.37873,97.4257,0.739534,99.802,0.358391,99.0099,0.039929,81.9802,94.5545
3,NVDA,230.090,8692,0.736149,83.5644,0.715856,99.604,0.259307,96.6337,0.11042,97.6238,94.3564
4,A,178.182,11224,0.750564,85.1485,0.408143,95.2475,0.207659,93.8614,0.095756,96.4356,92.6733
5,INTU,582.460,3433,0.802408,86.7327,0.458926,96.8317,0.226114,95.6436,0.0606672,91.0891,92.5743
6,NTAP,95.890,20857,1.12794,95.2475,0.375705,94.0594,0.118133,81.7822,0.139593,99.0099,92.5248
7,SNPS,340.300,5877,0.609278,77.0297,0.451502,96.6337,0.277043,97.0297,0.147547,99.2079,92.4752
8,KEYS,184.150,10860,0.874878,89.1089,0.318453,89.901,0.220409,95.0495,0.0820152,95.4455,92.3762
9,RMD,291.840,6853,0.696975,81.3861,0.575799,98.6139,0.236142,95.8416,0.059628,90.495,91.5842


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [63]:
writer = pd.ExcelWriter('momemtum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [64]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [65]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [66]:
writer.save()